Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [2]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [3]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

In [4]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [5]:
all_data.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


In [6]:
grid.head()

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [7]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


In [8]:
item_id_target_mean.head()

item_id
0    0.020000
1    0.023810
2    0.019802
3    0.019802
4    0.020000
Name: target, dtype: float64

In [9]:
test_data = all_data[:10]

In [10]:
test_data

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.022222
141495,0,27,0,0.0,0.056834
144968,0,28,0,0.0,0.141176
142661,0,29,0,0.0,0.037383
138947,0,32,0,6.0,1.319042
138948,0,33,0,3.0,0.527112
138949,0,34,0,0.0,0.146108
139247,0,35,0,1.0,0.944681
142672,0,40,0,0.0,0.070943
142065,0,41,0,0.0,0.085828


In [12]:
test_data['item_id']

139255    19
141495    27
144968    28
142661    29
138947    32
138948    33
138949    34
139247    35
142672    40
142065    41
Name: item_id, dtype: int32

In [13]:

test_data['item_id'].map(item_id_target_mean)

139255    0.022222
141495    0.056834
144968    0.141176
142661    0.037383
138947    1.319042
138948    0.527112
138949    0.146108
139247    0.944681
142672    0.070943
142065    0.085828
Name: item_id, dtype: float64

In [17]:
test_data.groupby('item_id')

In [19]:
test_data.groupby('item_id').target

#### Method 2

In [20]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


In [21]:
test_data.groupby('item_id').target.transform('mean')

139255    0.0
141495    0.0
144968    0.0
142661    0.0
138947    6.0
138948    3.0
138949    0.0
139247    1.0
142672    0.0
142065    0.0
dtype: float64

In [22]:
test_data.groupby('item_id').target.mean()

item_id
19    0.0
27    0.0
28    0.0
29    0.0
32    6.0
33    3.0
34    0.0
35    1.0
40    0.0
41    0.0
Name: target, dtype: float64

In [23]:
all_data['target'].mean()

0.33427305671234259

See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [24]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [83]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=False)
kf.get_n_splits(all_data)

# del all_data['item_target_enc']
df = all_data
all_data['item_target_enc'] = 0.3343

for tr_idx, test_idx in kf.split(df):
    print("train: {}, test: {}".format(len(tr_idx), len(test_idx)))
    other_folds, this_fold = df.iloc[tr_idx], df.iloc[test_idx]
    item_id_target_mean = other_folds.groupby('item_id').target.mean()
    all_data.item_target_enc.iloc[test_idx] = this_fold['item_id'].map(item_id_target_mean)

all_data['item_target_enc'].fillna(0.3343, inplace=True) 
all_data.head()
    

train: 8731080, test: 2182770


/Users/manuel/.pyenv/versions/3.5.2/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


train: 8731080, test: 2182770
train: 8731080, test: 2182770
train: 8731080, test: 2182770
train: 8731080, test: 2182770


,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.334300
141495,0,27,0,0.0,0.048523
144968,0,28,0,0.0,0.142424
142661,0,29,0,0.0,0.030303
138947,0,32,0,6.0,0.894020


In [84]:
# YOUR CODE GOES HERE

encoded_feature = all_data['item_target_enc'].values

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

0.41645907128
Current answer for task KFold_scheme is: 0.41645907128


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [98]:
all_data[:100000].groupby('item_id').target.agg(['sum','count']).head()

,sum,count
item_id,,
19,0.0,13
27,3.0,13
28,1.0,13
29,1.0,13
32,39.0,13


In [100]:
item_stats = all_data.groupby('item_id').target.agg(['sum','count'])
item_stats.head()

,sum,count
item_id,,
0,1.0,50
1,6.0,252
2,2.0,101
3,2.0,101
4,1.0,50


In [112]:
item_stats['sum'].head()

item_id
0    1.0
1    6.0
2    2.0
3    2.0
4    1.0
Name: sum, dtype: float64

In [111]:
test_data = all_data[:10]
(test_data.item_id.map(item_stats['sum']) - test_data.target) / (test_data.item_id.map(item_stats['count']) - 1)

139255    0.022727
141495    0.056911
144968    0.141414
142661    0.037500
138947    1.316088
138948    0.525552
138949    0.146283
139247    0.944444
142672    0.070997
142065    0.086000
dtype: float64

In [113]:
# YOUR CODE GOES HERE
item_stats = all_data.groupby('item_id').target.agg(['sum','count'])
encoded_feature = (all_data.item_id.map(item_stats['sum']) - all_data.target) / (all_data.item_id.map(item_stats['count']) - 1)

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.480384831129
Current answer for task Leave-one-out_scheme is: 0.480384831129


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [ ]:
# YOUR CODE GOES HERE

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [ ]:
# YOUR CODE GOES HERE

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [114]:
STUDENT_EMAIL = 'wesen@ruinwesen.com'# EMAIL HERE
STUDENT_TOKEN = '7DSHYVf3VeRdcvk5'# TOKEN HERE
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.41645907128
Task Leave-one-out_scheme: 0.480384831129
Task Smoothing_scheme: ----------
Task Expanding_mean_scheme: ----------


In [115]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
